In [1]:
import pylab
import pymoc
import xidplus
import numpy as np
%matplotlib inline
from astropy.table import Table

In [2]:
import seaborn as sns

This notebook uses all the raw data from the XID+MIPS catalogue, maps, PSF and relevant MOCs to create XID+ prior object and relevant tiling scheme

## Read in MOCs
The selection functions required are the main MOC associated with the masterlist. 

In [3]:
Sel_func=pymoc.MOC()
Sel_func.read('../data/CDFS-SWIRE/holes_CDFS-SWIRE_irac1_O16_MOC.fits')


## Read in XID+MIPS catalogue

In [4]:
XID_MIPS=Table.read('../data/CDFS-SWIRE/MIPS/dmu26_XID+MIPS_CDFS-SWIRE_cat_20170901.fits')

In [5]:
XID_MIPS[0:10]

help_id,RA,Dec,F_MIPS_24,FErr_MIPS_24_u,FErr_MIPS_24_l,Bkg_MIPS_24,Sig_conf_MIPS_24,Rhat_MIPS_24,n_eff_MIPS_24,Pval_res_24,flag_mips_24
,deg,deg,uJy,uJy,uJy,MJy / sr,MJy / sr,,,,
str27,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,bool
HELP_J033650.824-295620.783,54.21176639,-29.939106486,256.714,552.544,78.7135,-0.0034898,5.16096e-06,0.999126,1578.0,0.0,False
HELP_J033655.442-295449.596,54.2310102146,-29.9137765914,241.177,525.111,75.9436,-0.000860725,5.00049e-06,1.00131,1393.0,0.0,False
HELP_J033616.314-295548.181,54.0679762,-29.930050306,301.589,317.576,285.959,-0.00416415,5.00936e-06,nan,2000.0,0.0,False
HELP_J033601.094-295546.425,54.0045569,-29.929562436,81.0617,94.9617,68.2415,0.00167452,4.96993e-06,nan,1121.0,0.0,False
HELP_J033606.888-295518.496,54.0287013,-29.921804496,35.2795,49.9402,21.0285,-0.00524232,5.13793e-06,nan,1058.0,0.0,False
HELP_J033607.350-295516.790,54.03062359,-29.921330436,184.848,199.895,169.931,-0.00524232,5.13793e-06,nan,1555.0,0.0,False
HELP_J033608.786-295409.821,54.03660934,-29.902727956,7.16189,15.6523,1.85904,-0.00524232,5.13793e-06,nan,1605.0,0.0,True
HELP_J033610.856-295506.940,54.04523478,-29.918594406,250.447,262.422,237.689,-0.00524232,5.13793e-06,nan,1710.0,0.0,False


In [ ]:
skew=(XID_MIPS['FErr_MIPS_24_u']-XID_MIPS['F_MIPS_24'])/(XID_MIPS['F_MIPS_24']-XID_MIPS['FErr_MIPS_24_l'])
skew.name='(84th-50th)/(50th-16th) percentile'
g=sns.jointplot(x=np.log10(XID_MIPS['F_MIPS_24']),y=skew, kind='hex')


The uncertianties become Gaussian by $\sim 20 \mathrm{\mu Jy}$

In [6]:
good=XID_MIPS['F_MIPS_24']>20

In [7]:
good.sum()

240489

## Read in Maps

In [9]:

pswfits='../data/CDFS-SWIRE/SPIRE/CDFS-SWIRE-NEST_image_250_SMAP_v6.0.fits'#SPIRE 250 map
pmwfits='../data/CDFS-SWIRE/SPIRE/CDFS-SWIRE-NEST_image_350_SMAP_v6.0.fits'#SPIRE 350 map
plwfits='../data/CDFS-SWIRE/SPIRE/CDFS-SWIRE-NEST_image_500_SMAP_v6.0.fits'#SPIRE 500 map

#output folder
output_folder='./'

In [10]:
from astropy.io import fits
from astropy import wcs

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist[0].header
im250hdu=hdulist[1].header

im250=hdulist[1].data*1.0E3 #convert to mJy
nim250=hdulist[2].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist[1].header)
pixsize250=3600.0*w_250.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist[0].header
im350hdu=hdulist[1].header

im350=hdulist[1].data*1.0E3 #convert to mJy
nim350=hdulist[2].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist[1].header)
pixsize350=3600.0*w_350.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist[0].header
im500hdu=hdulist[1].header
im500=hdulist[1].data*1.0E3 #convert to mJy
nim500=hdulist[2].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist[1].header)
pixsize500=3600.0*w_500.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()

In [14]:
## Set XID+ prior class

In [15]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu, moc=Sel_func)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(XID_MIPS['RA'][good],XID_MIPS['Dec'][good],'dmu26_XID+MIPS_CDFS-SWIRE_cat_20170901.fits',ID=XID_MIPS['help_id'][good])#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu, moc=Sel_func)
prior350.prior_cat(XID_MIPS['RA'][good],XID_MIPS['Dec'][good],'dmu26_XID+MIPS_CDFS-SWIRE_cat_20170901.fits',ID=XID_MIPS['help_id'][good])
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu, moc=Sel_func)
prior500.prior_cat(XID_MIPS['RA'][good],XID_MIPS['Dec'][good],'dmu26_XID+MIPS_CDFS-SWIRE_cat_20170901.fits',ID=XID_MIPS['help_id'][good])
prior500.prior_bkg(-5.0,5)

In [16]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [17]:
import pickle
#from moc, get healpix pixels at a given order
from xidplus import moc_routines
order=9
tiles=moc_routines.get_HEALPix_pixels(order,prior250.sra,prior250.sdec,unique=True)
order_large=6
tiles_large=moc_routines.get_HEALPix_pixels(order_large,prior250.sra,prior250.sdec,unique=True)
print('----- There are '+str(len(tiles))+' tiles required for input catalogue and '+str(len(tiles_large))+' large tiles')
output_folder='./'
outfile=output_folder+'Master_prior.pkl'
with open(outfile, 'wb') as f:
    pickle.dump({'priors':[prior250,prior350,prior500],'tiles':tiles,'order':order,'version':xidplus.io.git_version()},f)
outfile=output_folder+'Tiles.pkl'
with open(outfile, 'wb') as f:
    pickle.dump({'tiles':tiles,'order':order,'tiles_large':tiles_large,'order_large':order_large,'version':xidplus.io.git_version()},f)
raise SystemExit()

----- There are 670 tiles required for input catalogue and 18 large tiles


SystemExit: 

In [18]:
prior250.nsrc

240489